In [1]:
from src.recommenders import ImprovedRecommender, PopBasedRecommender
from sklearn.decomposition import TruncatedSVD

In [2]:
qual_eval_folder = './evaluation'
items_path = "./data/games_original.pkl"
data_path = "./data/interactions_split_0"

From the base recommender results, we will now continue to use the ideal combination of methods as a baseline before we make further improvements.
This means we use cosine distance and normalized feature vectors. We also don't use feedback weighting.

The following cell runs the basic recommender and provides more in-depth evaluation and compares it to a popularity-based recommender.

In [8]:
rec = PopBasedRecommender(train_path=f"{data_path}_train.parquet", test_path=f"{data_path}_test.parquet", val_path=f"{data_path}_val.parquet")
rec.generate_recommendations(read_max=1000)
print(rec.evaluate(k=10))
dim_red = TruncatedSVD(n_components=100, random_state=500)
rec = ImprovedRecommender(items_path, train_path=f"{data_path}_train.parquet", test_path=f"{data_path}_test.parquet", val_path=f"{data_path}_val.parquet", sparse=True, tfidf='smooth', use_feedback=False, normalize=True, dim_red=None)
rec.generate_recommendations(read_max=1000)
rec.evaluate(k=10)

{'HR@10': 0.915, 'nDCG@10': 0.29887349314420036, 'recall@10': 0.051815443190272176, 'ideal_recall@10': 0.18387381026842325, 'nRecall@10': 0.2865}


1000it [08:32,  1.95it/s]


{'HR@10': 0.717,
 'nDCG@10': 0.13052286421550363,
 'recall@10': 0.023317434904203696,
 'ideal_recall@10': 0.18387381026842325,
 'nRecall@10': 0.1292}

In [4]:
import pandas as pd
import re
import numpy as np
items = pd.read_pickle(items_path)
items["price"] = items["price"].apply(lambda p: np.float32(p) if re.match(r"\d+(?:.\d{2})?", str(p)) else 0)
items["metascore"] = items["metascore"].apply(lambda m: m if m != "NA" else np.nan)
items["developer"].fillna(value='', inplace=True)
items["developer"] = items["developer"].apply(lambda my_str: my_str.lower().split(','))
items["publisher"].fillna(value='', inplace=True)
items["publisher"] = items["publisher"].apply(lambda my_str: my_str.lower().split(','))
items["early_access"] = items["early_access"].apply(lambda x: ["earlyaccess"] if x else [])
items["specs"] = items["specs"].apply(lambda l: [re.subn(r"[^a-z0-9]", "", my_str.lower())[0] for my_str in l])
items["tags"] = items["tags"].apply(lambda l: [re.subn(r"[^a-z0-9]", "", my_str.lower())[0] for my_str in l])
items["genres"] = items["genres"].apply(lambda l: [re.subn(r"[^a-z0-9]", "", my_str.lower())[0] for my_str in l])

items

,publisher,genres,app_name,release_date,tags,discount_price,specs,price,early_access,id,developer,sentiment,metascore,users_count
0,[valve],[action],Counter-Strike: Global Offensive,2012-08-21,"[fps, multiplayer, shooter, action, teambased,...",NaN,"[multiplayer, steamachievements, fullcontrolle...",14.99,[],730,[valve],Very Positive,83.0,42620
1,[valve],"[indie, simulation]",Garry's Mod,2006-11-29,"[sandbox, multiplayer, funny, moddable, buildi...",NaN,"[singleplayer, multiplayer, coop, crossplatfor...",9.99,[],4000,[facepunch studios],Overwhelmingly Positive,NaN,42157
2,[smartly dressed games],"[action, adventure, casual, freetoplay, indie]",Unturned,2017-07-07,"[freetoplay, survival, zombies, multiplayer, o...",NaN,"[singleplayer, onlinemultiplayer, onlinecoop, ...",0.00,[],304930,[smartly dressed games],Very Positive,NaN,37654
3,[valve],[action],Left 4 Dead 2,2009-11-16,"[zombies, coop, fps, multiplayer, action, onli...",NaN,"[singleplayer, multiplayer, coop, steamachieve...",19.99,[],550,[valve],Overwhelmingly Positive,89.0,35990
4,[re-logic],"[action, adventure, indie, rpg]",Terraria,2011-05-16,"[sandbox, adventure, survival, 2d, multiplayer...",NaN,"[singleplayer, multiplayer, onlinemultiplayer,...",9.99,[],105600,[re-logic],Overwhelmingly Positive,83.0,28551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7345,[projectile entertainment],[indie],Momentum,2016-08-11,"[indie, physics]",NaN,"[singleplayer, steamachievements, fullcontroll...",9.99,[],462130,[projectile entertainment],8 user reviews,NaN,1
7346,[nuligine],"[action, indie, racing, simulation]",MadOut Ice Storm,2015-09-04,"[racing, indie, action, simulation]",NaN,"[singleplayer, fullcontrollersupport]",6.99,[],400500,[nuligine],Mixed,NaN,1
7347,[like a boss llc],"[action, adventure, indie, strategy]",OCCHIO,2016-08-11,"[action, adventure, indie, strategy, puzzlepla...",NaN,"[singleplayer, fullcontrollersupport]",1.99,[],513140,[acerio games],7 user reviews,NaN,1
7348,[dxf games],"[action, indie, rpg]",Just a Cleric,2016-05-19,"[rpg, indie, action, pixelgraphics, difficult]",NaN,"[singleplayer, steamachievements, steamtrading...",4.99,[],467330,[dxf games],Positive,NaN,1
